<a href="https://colab.research.google.com/github/min207/2023-1-ESAA/blob/main/%EB%B6%84%EB%A5%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [7]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False)

In [8]:
X, y = mnist["data"], mnist["target"]

In [9]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다. 
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [10]:
# import package
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [12]:
# Try GridSearch to optimize hyperparameter
param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [4, 5]}]

knn_clf = KNeighborsClassifier()
grid = GridSearchCV(knn_clf, param_grid, cv=3)
grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [4, 5],
                          'weights': ['uniform', 'distance']}])

In [13]:
# best hyperparameter
grid.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [14]:
# best score
grid.best_score_

0.9703500000000002

In [15]:
# model test
from sklearn.metrics import accuracy_score
y_pred = grid.predict(X_test)
accuracy_score(y_test, y_pred)

0.9714

### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

In [16]:
from scipy.ndimage.interpolation import shift

<ipython-input-16-c50379b1544a>:1: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift


In [17]:
def shift_image(image, x, y):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [y, x], cval=0, mode="constant")
    return shifted_image.reshape([-1])

####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [18]:
X_train_2 = [image for image in X_train]
y_train_2 = [label for label in y_train]

for x, y in ((1, 0), (-1, 0), (0, 1), (0, -1)):
  for image, label in zip(X_train, y_train):
    X_train_2.append(shift_image(image, x, y))
    y_train_2.append(label)

X_train_2 = np.array(X_train_2)
y_train_2 = np.array(y_train_2)

In [27]:
shuffle_idx = np.random.permutation(len(X_train_2))
X_train_2 = X_train_2[shuffle_idx]
y_train_2 = y_train_2[shuffle_idx]

####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [28]:
knn_clf_2 = KNeighborsClassifier(n_neighbors = 4, weights = 'distance')

In [29]:
knn_clf_2.fit(X_train_2, y_train_2)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [31]:
y_pred_2 = knn_clf_2.predict(X_test)
accuracy_score(y_test, y_pred_2)

0.9763